<a href="https://colab.research.google.com/github/meozbrls/Flight_Prices_Analysis/blob/main/notebooks/flight_prices_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Flight Prices Analysis

In [14]:
!pip -q install gdown pyarrow

import gdown, os
import pandas as pd

FILE_ID = "15MB8kYT9Ny2_CJ_SvFvPKTSBkmfUL0wv"   # linkten
OUT_PATH = "data/flightprices_2022_06.parquet"  # local çalışma yolu

os.makedirs("data", exist_ok=True)

if not os.path.exists(OUT_PATH):
    url = f"https://drive.google.com/uc?id={FILE_ID}"
    gdown.download(url, OUT_PATH, quiet=False)

df = pd.read_parquet(OUT_PATH)
df.head()


,legId,searchDate,flightDate,startingAirport,destinationAirport,fareBasisCode,travelDuration,elapsedDays,isBasicEconomy,isRefundable,...,baseFare,totalFare,seatsRemaining,totalTravelDistance,segmentsDepartureTimeRaw,segmentsArrivalTimeRaw,segmentsAirlineName,segmentsAirlineCode,segmentsEquipmentDescription,segmentsCabinCode
0,684e879e0f2cefcdbfba1f7e35f897a3,2022-04-17,2022-06-01,ATL,BOS,RA7NR,PT2H28M,0,False,False,...,31.00,87.59,0,NaN,2022-06-01T06:25:00.000-04:00,2022-06-01T08:53:00.000-04:00,Spirit Airlines,NK,Airbus A319,coach
1,222cfd6d1b0d5732602a3e82ad7730c3,2022-04-17,2022-06-01,ATL,BOS,W21ZXP5,PT4H50M,0,False,False,...,65.48,93.99,4,947.0,2022-06-01T06:00:00.000-04:00||2022-06-01T09:3...,2022-06-01T08:05:00.000-04:00||2022-06-01T10:5...,Frontier Airlines||Frontier Airlines,F9||F9,Airbus A320||,coach||coach
2,5b13b222dff8d227c34ba5f0b10a8b5a,2022-04-17,2022-06-01,ATL,BOS,GA7NR,PT8H38M,1,False,False,...,25.58,95.16,0,NaN,2022-06-01T15:29:00.000-04:00||2022-06-01T20:5...,2022-06-01T17:28:00.000-04:00||2022-06-02T00:0...,Spirit Airlines||Spirit Airlines,NK||NK,||,coach||coach
3,71cf5163f5efbd007c87aeef85e0c2cc,2022-04-17,2022-06-01,ATL,BOS,UA3NX0MC,PT2H37M,0,False,False,...,161.86,188.60,9,947.0,2022-06-01T12:58:00.000-04:00,2022-06-01T15:35:00.000-04:00,Delta,DL,None,coach
4,141ef83862caac6be402158433b55c1f,2022-04-17,2022-06-01,ATL,BOS,UAUOA0MC,PT2H41M,0,False,False,...,161.86,188.60,2,947.0,2022-06-01T16:10:00.000-04:00,2022-06-01T18:51:00.000-04:00,Delta,DL,Airbus A321,coach


In [15]:
df.shape

(11774933, 21)

In [16]:
# Kolon tiplerini analiz ediyoruz
dtype_table = pd.DataFrame({
    "column": df.columns,
    "dtype": df.dtypes.values,
    "non_null": df.notnull().sum().values,          # boş olmayan kayıt sayısı
    "null_ratio": (df.isnull().mean().values * 100).round(2),  # null oranı (%)
    "unique_values": df.nunique().values            # benzersiz değer sayısı
})

dtype_table.sort_values(by="dtype")


,column,dtype,non_null,null_ratio,unique_values
10,isNonStop,bool,11774933,0.00,2
2,flightDate,datetime64[ns],11774933,0.00,30
8,isBasicEconomy,bool,11774933,0.00,2
9,isRefundable,bool,11774933,0.00,2
7,elapsedDays,int64,11774933,0.00,3
13,seatsRemaining,int64,11774933,0.00,11
14,totalTravelDistance,float64,10797713,8.30,2613
11,baseFare,float64,11774933,0.00,22324
12,totalFare,float64,11774933,0.00,38484
18,segmentsAirlineCode,object,11774933,0.00,116


In [17]:
# searchDate kolonunu zaman tipine dönüştürdük.
df["searchDate"] = pd.to_datetime(df["searchDate"], errors="coerce")

# Dönüşüm sonrası dtype'lar
print(df[["searchDate"]].dtypes)
print("searchDate NaT sayısı:", df["searchDate"].isna().sum())
print("flightDate NaT sayısı:", df["flightDate"].isna().sum())

searchDate    datetime64[ns]
dtype: object
searchDate NaT sayısı: 0
flightDate NaT sayısı: 0


In [18]:
# Sayısal kolonların özet istatistikleri.
num_cols = ["baseFare","totalFare","seatsRemaining","totalTravelDistance","elapsedDays"]
display(df[num_cols].describe(include="all").T)


,count,mean,std,min,25%,50%,75%,max
baseFare,11774933.0,348.019151,205.094231,2.00,198.14,320.0,462.32,7662.33
totalFare,11774933.0,400.161910,218.843954,29.59,238.60,368.6,521.01,8260.61
seatsRemaining,11774933.0,5.546237,2.969681,0.00,3.00,7.0,7.00,10.00
totalTravelDistance,10797713.0,1558.916426,843.709368,97.00,848.00,1392.0,2377.00,4681.00
elapsedDays,11774933.0,0.154232,0.361225,0.00,0.00,0.0,0.00,2.00
